In [2]:
"""This is video reader with all operations inside fnctions, so they can be called on 
    each frame out of a set of frames, and each video out of a set of videos
    """

'This is video reader with all operations inside fnctions, so they can be called on \n    each frame out of a set of frames, and each video out of a set of videos\n    '

In [8]:
"""Imports"""
import matplotlib.pyplot as plt #for maknig plots inside the notebook
import skimage
import skvideo.io
from skvideo.io import vreader, ffprobe
from skimage import measure, morphology, feature
from skimage.filters import *
from skimage.morphology import *
from operator import attrgetter
import numpy as np
import math
from itertools import chain
#from functools import partial, update_wrapper
from scipy import ndimage 
from pims import pipeline, Video
from glob import glob
import seaborn as sns
from collections import OrderedDict 
#import antigravity

In [13]:
"""Functions"""
# apply total thresholding to each of a list of frames; filter slides with wrong number

def total_threshold_filter(frame, frame_no, transition_threshold, class_thresh,
                           broken_count, class_num, origin, last_class, last_whole, filtrate_len, 
                           sides, classes, transitions): 
    
   
    
   # ---setup image and detect blobs ----------------
    thresh_img = frame > threshold_isodata(frame)# binary image
    highlight = morphology.opening(thresh_img, square(7))
    img = np.copy(frame)
    img[highlight==0] = 0
    blobs =skimage.feature.blob_doh(img, min_sigma = 6, max_sigma = 16, threshold = 0.007, num_sigma= 15, overlap=0.8)

    num_connections = 0

    blobs = sorted(blobs, key = third_item, reverse=True)
    num_blobs = len(blobs)
    if num_blobs > 0:
        max_blob = blobs[0]
        connect_dists =connect((max_blob[1],max_blob[0]),max_rad, blobs[1::])
        cons_of_max = connect_dists[0]
        distances = connect_dists[1]        
        num_connections +=cons_of_max 
        
        length_of_transparents=0
        num_smaller_blobs = num_blobs - 1
        for i in range(num_smaller_blobs-1):
            blob = blobs[i+1]
            center_vec = (blob[1], blob[0])
            connect_dists = connect(center_vec, max_rad, blobs[i+2::])
            num_connections += connect_dists[0]
            length_of_transparents += connect_dists[1]
    
    #----------------------filter------------------------- can use actual filter
    test = (num_blobs == expected_blobs
                and num_connections == expected_connections 
               )
    if test: # keep frames that have enough paricles, and are not transitions
        side =sideify(cons_of_max, length_of_transparents )
        clas = side
        sides[side].append(frame_no)
        classes[clas].append(frame_no)
            #print(clas)
        
        if clas == 'ucf':
            #broken_count += 1
            #class_num = 0
            #print('ucf')
            pass
        else:
            if origin == 'ucf':
                origin = last_class
                #print("reset origin", origin)
            if clas == last_class:
                class_num += 1
                #print("class_num: %s" %class_num)
            else:
                class_num = 0
                #print("class_reset")
                #print("class_num: %s" %class_num)
            if class_num >= class_thresh: 
                if broken_count >= transition_threshold or clas != origin:
                    transitions[origin + "->" + clas].append((last_whole, frame_no))
                    #print(origin + "->" + clas)
                    origin = clas
                    #print("origin: %s" %origin)
                broken_count = 0
                #print("broken_count: %s" %broken_count)
                
            
                
            last_whole = frame_no
            filtrate_len += 1 
            last_class = clas
            
            
    else:
        #print("broken")
        class_num = 0
        #print("class_num: %s" %class_num)
        broken_count += 1
        #print("broken_count: %s" %broken_count)

    return [broken_count, class_num, origin, last_class, last_whole, filtrate_len]



        
# take largest isodata image region, and return aclassification
def sideify(cons_of_max, lens):    
    if cons_of_max == 3 and lens < 115 and lens > 98:
        return "c"
    elif cons_of_max == 2 and lens < 95 and lens > 80:
        return "t"
    else:
        return "ucf"

def connect(cv, radius, bloblist): # center of blob2, radius of all blobs, list of other blobs
    num_connections = 0
    length = 0
    for blob2 in bloblist:
        cv2 = (blob2[1], blob2[0]) # center of blob2
        vd = vector_dist(cv, cv2)
        length += vd
        if vd <= 2*radius: # blob centers closer than diameter of one blob
            num_connections += 1
        else:
            pass
    return (num_connections, length)

def vector_dist(v1, v2): # euclidean distance between 2 points
    return math.sqrt(np.sum([(v1[i] - v2[i])**2 for i in range(len(v1))]))

def third_item(l1):
        return l1[2]   

In [ ]:
"""Pipeline for lists of frames"""
""""""
"""Pipeline for videos:"""
# initialize frame lists
filenames = glob("./tchakamau/trn*/*[1][0]v*[7].avi")#["./tchakamau/New_setup/45681_10Vpp_25.avi"
print(filenames)
num_top_keys = 3
num_full_keys = 3
num_tran_keys = 2
trans_threshes = np.linspace(5, 100, 20)#[30, 40, 50]#[5, 8, 10, 19, 27, 38, 52, 60]
class_threshes = np.linspace(5, 100, 20)#[15, 22, 30]
trans_runs = {}
#trans_fil = 10

ave = np.average

expected_blobs = 4
expected_connections = 5
max_rad = 15*1.35
"""area_threshold = 1800   # must have enough particles
convex_thresholds = (2000, 2800)   # convex hull area (> 2500 filtered)  # particles must be in parallelogram
minor_thresholds = (38, 48) # minor_axis length (< filterd) # particles must be in parallelogram
major_thresholds = (59, 76)"""
thresholds = [expected_blobs , expected_connections, max_rad ]
params = [3, 115, 98, 2, 95, 80]
print("Number files: ", len(filenames))
index = 0
for trans_fil in trans_threshes:
    
    for class_thresh in class_threshes:
        
        Top_bottoms = np.zeros((len(filenames), num_top_keys))
        Full_classif = np.zeros((len(filenames), num_full_keys))
        N_transitions = np.zeros((len(filenames), num_tran_keys, num_tran_keys))
        num_total_frames = 0 
        filtrate_len = 0
        for vidnum in range(len(filenames)):
            print("Processing vid %s : %s" %(vidnum, filenames[vidnum]))
            #meta = ffprobe(filenames[vidnum])
            #print(meta)#['@nb_frames'])
            #print(5)
            frames = vreader(filenames[vidnum])
            #frame_vid = frames[:, :, :, 2]# making videos ino a frame list
            #print(frames)



            sides = {"t":[], "c":[], "ucf":[]}
            classes = {"t":[], "c":[], "ucf":[]}
            transitions = [("t->t",[]), ("t->c",[]),
                          ("c->t" ,[]), ("c->c",[])]

            transitions = OrderedDict(transitions)
            broken_count = 0
            class_num = 0
            origin = 'ucf'
            last_class = 'ucf'
            last_whole = 0
            for fr in frames:
                frame = fr[:, :, 2]
                num_total_frames += 1
                org = total_threshold_filter(frame, num_total_frames, trans_fil, class_thresh,
                                             broken_count, class_num, origin, last_class, last_whole, filtrate_len, 
                                             sides, classes, transitions)
                broken_count, class_num, origin, last_class, last_whole, filtrate_len = org
            filtered_len = filtrate_len
            #----------------------------------------------------------------
            trans_len = len([y for x in transitions.values() for y in x])
            skeys = list(sides.keys())
            ckeys = list(classes.keys())
            tkeys = list(transitions.keys())
            slcs = [len(sides[x]) for x in skeys]
            clcs = [len(classes[x]) for x in ckeys]
            trans = [len(transitions[x]) for x in tkeys] #num of each transition
            ucf = len(sides["ucf"])


            Top_bottoms[vidnum,:] = [slcs[i] for i in range(len(skeys))]
            Full_classif[vidnum,:] = [clcs[i] for i in range(len(ckeys))]
            N_transitions[vidnum,:] = np.reshape(trans, (num_tran_keys, num_tran_keys))
            print("T threshold: %s" %trans_fil)
            print("C threshold: %s" %class_thresh)
            print(N_transitions[vidnum,:])
            #print("finished vid %s" %vidnum)
        trans_runs[index] = [[Top_bottoms, Full_classif, N_transitions, (trans_fil, class_thresh), num_top_keys, num_full_keys,
                                 num_tran_keys, skeys, ckeys, num_tran_keys, num_total_frames], thresholds, params]
        index += 1
        #---------------------------------------------------------------------------------

#-----------------------------------------------------------------------

np.save('trans_runs_transparentd_err.npy',trans_runs)


['./tchakamau\\Trnansparentd\\45403_10Vpp_7.avi']
Number files:  1
Processing vid 0 : ./tchakamau\Trnansparentd\45403_10Vpp_7.avi
T threshold: 5.0
C threshold: 5.0
[[98.  8.]
 [ 7. 77.]]
Processing vid 0 : ./tchakamau\Trnansparentd\45403_10Vpp_7.avi
T threshold: 5.0
C threshold: 10.0
[[83.  7.]
 [ 6. 50.]]
Processing vid 0 : ./tchakamau\Trnansparentd\45403_10Vpp_7.avi
T threshold: 5.0
C threshold: 15.0
[[79.  7.]
 [ 6. 46.]]
Processing vid 0 : ./tchakamau\Trnansparentd\45403_10Vpp_7.avi


In [ ]:
"""Pipeline for lists of frames"""
""""""
"""Pipeline for videos:"""
# initialize frame lists
filenames = glob("./tchakamau/2ND*/*7.avi")#["./tchakamau/New_setup/45681_10Vpp_25.avi"
print(filenames)
num_top_keys = 3
num_full_keys = 3
num_tran_keys = 2
trans_threshes = [25, 30, 50]#[5, 8, 10, 19, 27, 38, 52, 60]
trans_runs = {}
#trans_fil = 10

ave = np.average

area_threshold = 1850   # must have enough particles
sol_threshold = 0.95
iner_threshold = (320, 410)
convex_thresholds = (2000, 2800)   # convex hull area (> 2500 filtered)  # particles must be in parallelogram
minor_thresholds = (38, 48) # minor_axis length (< filterd) # particles must be in parallelogram
#major_thresholds = (59, 76)
major_threshold = 78
"""area_threshold = 1800   # must have enough particles
convex_thresholds = (2000, 2800)   # convex hull area (> 2500 filtered)  # particles must be in parallelogram
minor_thresholds = (38, 48) # minor_axis length (< filterd) # particles must be in parallelogram
major_thresholds = (59, 76)"""

print("Number files: ", len(filenames))

for trans_fil in trans_threshes:
    print("T threshold: %s" %trans_fil)
    Top_bottoms = np.zeros((len(filenames), num_top_keys))
    Full_classif = np.zeros((len(filenames), num_full_keys))
    N_transitions = np.zeros((len(filenames), num_tran_keys, num_tran_keys))
    num_total_frames = 0 
    filtrate_len = 0
    for vidnum in range(len(filenames)):
        print("Processing vid %s : %s" %(vidnum, filenames[vidnum]))
        #meta = ffprobe(filenames[vidnum])
        #print(meta)#['@nb_frames'])
        #print(5)
        frames = vreader(filenames[vidnum])
        #frame_vid = frames[:, :, :, 2]# making videos ino a frame list
        #print(frames)



        sides = {"t":[], "c":[], "ucf":[]}
        classes = {"t":[], "c":[], "ucf":[]}
        transitions = [("t->t",[]), ("t->c",[]),
                      ("c->t" ,[]), ("c->c",[])]
        
        transitions = OrderedDict(transitions)
        broken_count = 0
        class_num = 0
        origin = 'ucf'
        last_class = 'ucf'
        last_whole = 0
        for fr in frames:
            frame = fr[:, :, 2]
            num_total_frames += 1
            org = total_threshold_filter(frame, num_total_frames, trans_fil,
                                         broken_count, class_num, origin, last_class, last_whole, filtrate_len, 
                                         sides, classes, transitions)
            broken_count, class_num, origin, last_class, last_whole, filtrate_len = org
        filtered_len = filtrate_len
        #----------------------------------------------------------------
        trans_len = len([y for x in transitions.values() for y in x])
        skeys = list(sides.keys())
        ckeys = list(classes.keys())
        tkeys = list(transitions.keys())
        slcs = [len(sides[x]) for x in skeys]
        clcs = [len(classes[x]) for x in ckeys]
        trans = [len(transitions[x]) for x in tkeys] #num of each transition
        ucf = len(sides["ucf"])


        Top_bottoms[vidnum,:] = [slcs[i] for i in range(len(skeys))]
        Full_classif[vidnum,:] = [clcs[i] for i in range(len(ckeys))]
        N_transitions[vidnum,:] = np.reshape(trans, (num_tran_keys, num_tran_keys))
        print(N_transitions[vidnum,:])
        #print("finished vid %s" %vidnum)
    trans_runs[trans_fil] = [Top_bottoms, Full_classif, N_transitions, trans_threshes, num_top_keys, num_full_keys,
                             num_tran_keys, skeys, ckeys, num_tran_keys, num_total_frames]
    #---------------------------------------------------------------------------------
    
#-----------------------------------------------------------------------

np.save('trans_runs_new1.npy',trans_runs)


In [ ]:
trans_runs= np.load('trans_runs2.npy')[()]
"""num_top_keys = 4
num_full_keys = 6
num_tran_keys = 5
trans_threshes = [5, 8, 10, 19, 27, 38, 52, 60]
sides = {"bm":[], "b":[], "t":[], "ucf":[]}
classes = {"bm":[], "br":[], "bl":[], "tr":[], "tl":[], "ucf":[]}
transitions = {"bm->bm" :[], "bm->br":[], "bm->bl":[], "bm->tr":[], "bm->tl":[],
              "br->bm" :[], "br->br":[], "br->bl":[], "br->tr":[], "br->tl":[],
              "bl->bm" :[], "bl->br":[], "bl->bl":[], "bl->tr":[], "bl->tl":[],
              "tr->bm" :[], "tr->br":[], "tr->bl":[], "tr->tr":[], "tr->tl":[],
              "tl->bm" :[], "tl->br":[], "tl->bl":[], "tl->tr":[], "tl->tl":[]}"""

skeys = list(sides.keys())
ckeys = list(classes.keys())
for trans_fil in trans_threshes:
    print("\nTRANSITION THRESHOLD ", trans_fil)
    
    #Top_bottoms, Full_classif, N_transitions = trans_runs[trans_fil]
    Top_bottoms, Full_classif, N_transitions, trans_threshes, num_top_keys, num_full_keys, num_tran_keys, skeys, ckeys, num_tran_keys, num_total_frames = trans_runs[trans_fil]
    sums = np.sum
    Tb_sums = [sums(Top_bottoms[:, c]) for c in range(num_top_keys)]
    Fc_sums = [sums(Full_classif[:, c]) for c in range(num_full_keys)]
    Nt_sums = sums(N_transitions, axis = 0)#[sums(N_transitions[:, c]) for c in range(num_tran_keys)]
    #print("\nNumber of frames obbserved: ", num_total_frames)

    Tb_percs = [Tb_sums[c] *100 /sums(Tb_sums) for c in range(num_top_keys)]
    Fc_percs = [Fc_sums[c] *100/sums(Fc_sums) for c in range(num_full_keys)]
    Nt_percs = Nt_sums *100/sums(Nt_sums)
    #print(sums(N_transitions), sums(Nt_sums))


    print("\nsums in broad classifications")
    [print(skeys[i], "sum %.f" %Tb_sums[i]) for i in range(len(skeys))]
    print("Total: ", sums(Tb_sums) )
    print("\nsums in narrow classifications")
    [print(ckeys[i], "sum %.f" %Fc_sums[i]) for i in range(len(ckeys))]
    print("Total: ", sums(Fc_sums ) )
    print("\nsums of transitions broad")
    T_eb = np.vstack([Nt_sums[0], 
                                 (Nt_sums[1] + Nt_sums[2]) , 
                                 (Nt_sums[3] + Nt_sums[4])])
    Nt_sums_broad = np.column_stack([T_eb[:, 0], 
                                 (T_eb[:, 1]+ T_eb[:, 2]), 
                                 (T_eb[:, 3]+ T_eb[:, 4])])
    print(Nt_sums_broad)
    print("\nsums of transitions narrow")
    print(Nt_sums)
    print("Total: ", sums(Nt_sums) )



    """print("\n%s in broad classifications")
    [print(skeys[i], " %.f%%" %Tb_percs[i]) for i in range(len(skeys))]
    print("\n%s in narrow classifications")
    [print(ckeys[i], " %.f%%" %Fc_percs[i]) for i in range(len(ckeys))]"""
    #ERROR ANALSIS HERE
    print("\nErrors in transition matrix ((row_sums - column_sums) / 2)")
    print(((sums(Nt_sums, axis = 1) - sums(Nt_sums, axis =0))/2))
    print("\nError %%s in transition matrix ((row_sums - column_sums) *100 / (row_sums + column_sums))")
    print(((sums(Nt_sums, axis = 1) - sums(Nt_sums, axis =0))*100 /(sums(Nt_sums, axis = 1) + sums(Nt_sums, axis =0))))
    
    print("\nTransition matrix, broad")
    T_eb = np.vstack([Nt_percs[0], 
                                 (Nt_percs[1] + Nt_percs[2]) , 
                                 (Nt_percs[3] + Nt_percs[4])])
    Nt_percs_broad = np.column_stack([T_eb[:, 0], 
                                 (T_eb[:, 1]+ T_eb[:, 2]), 
                                 (T_eb[:, 3]+ T_eb[:, 4])])
    print(Nt_percs_broad)
    """print("row_sums of transitin matrix, broad")
    print(sums(Nt_percs_broad, axis = 1))
    print("\n%s of transitions broad(->bm, ->b, ->t):")
    #print("->bm %.f%%" %(Nt_percs[0, 0] + sums(Nt_percs[1:3, 0]) + sums(Nt_percs[3:5, 0])))
    print([sums(Nt_sums, axis = 0)[0]*100/sums(Nt_sums), 
           (sums(Nt_sums, axis = 0)[1] + sums(Nt_sums, axis = 0)[2])*100/(sums(Nt_sums)),
           (sums(Nt_sums, axis = 0)[3] + sums(Nt_sums, axis = 0)[4])*100/(sums(Nt_sums))])
    print("\n%s of transitions broad(bm->, b->, t->):")
    #print("->bm %.f%%" %(Nt_percs[0, 0] + sums(Nt_percs[1:3, 0]) + sums(Nt_percs[3:5, 0])))
    print([sums(Nt_sums, axis = 1)[0]*100/sums(Nt_sums),
           (sums(Nt_sums, axis = 1)[1] + sums(Nt_sums, axis = 1)[2])*100/(sums(Nt_sums)),
           (sums(Nt_sums, axis = 1)[3] + sums(Nt_sums, axis =1)[4])*100/(sums(Nt_sums))])#(Nt_percs[i,1] + Nt_percs[i,1+1] + Nt_percs[i+1, 1] + Nt_percs[i+1, 1+1] + sums(Nt_percs[0, 1:3]))) for i in [1,3]]
    """
    #print("->t %.f%%" %(Nt_percs[i,2] + Nt_percs[i,2+1] + Nt_percs[i+1, 2] + Nt_percs[i+1, 2+1] + sums(Nt_percs[0, 3:5]))) for i in [1,3]]
    print("\nTransition matrix, narrow")
    print(Nt_percs)
    print(sums(Nt_percs))
    print("\n%s of transitions to states narrow:")
    [print(sums(Nt_percs, axis = 0)*100/sums(Nt_percs))]
    print("\n%s of transitions from states narrow:")
    [print(sums(Nt_percs, axis = 1)*100/sums(Nt_percs))]
    print("\n%s of transitions to states broad:")
    [print(sums(Nt_percs_broad, axis = 0)*100/sums(Nt_percs_broad))]
    print("\n%s of transitions from states broad:")
    [print(sums(Nt_percs_broad, axis = 1)*100/sums(Nt_percs_broad))]
    #print("\n%s of transitions narrow")

In [ ]:
trans_runs= np.load('trans_runs1.npy')[()]
"""num_top_keys = 4
num_full_keys = 6
num_tran_keys = 5
trans_threshes = [5, 8, 10, 19, 27, 38, 52, 60]
sides = {"bm":[], "b":[], "t":[], "ucf":[]}
classes = {"bm":[], "br":[], "bl":[], "tr":[], "tl":[], "ucf":[]}
transitions = {"bm->bm" :[], "bm->br":[], "bm->bl":[], "bm->tr":[], "bm->tl":[],
              "br->bm" :[], "br->br":[], "br->bl":[], "br->tr":[], "br->tl":[],
              "bl->bm" :[], "bl->br":[], "bl->bl":[], "bl->tr":[], "bl->tl":[],
              "tr->bm" :[], "tr->br":[], "tr->bl":[], "tr->tr":[], "tr->tl":[],
              "tl->bm" :[], "tl->br":[], "tl->bl":[], "tl->tr":[], "tl->tl":[]}"""

skeys = list(sides.keys())
ckeys = list(classes.keys())
for trans_fil in trans_threshes:
    print("\nTRANSITION THRESHOLD ", trans_fil)
    
    #Top_bottoms, Full_classif, N_transitions = trans_runs[trans_fil]
    Top_bottoms, Full_classif, N_transitions, trans_threshes, num_top_keys, num_full_keys, num_tran_keys, skeys, ckeys, num_tran_keys, num_total_frames = trans_runs[trans_fil]
    sums = np.sum
    Tb_sums = [sums(Top_bottoms[:, c]) for c in range(num_top_keys)]
    Fc_sums = [sums(Full_classif[:, c]) for c in range(num_full_keys)]
    Nt_sums = sums(N_transitions, axis = 0)#[sums(N_transitions[:, c]) for c in range(num_tran_keys)]
    #print("\nNumber of frames obbserved: ", num_total_frames)

    Tb_percs = [Tb_sums[c] *100 /sums(Tb_sums) for c in range(num_top_keys)]
    Fc_percs = [Fc_sums[c] *100/sums(Fc_sums) for c in range(num_full_keys)]
    Nt_percs = Nt_sums *100/sums(Nt_sums)
    #print(sums(N_transitions), sums(Nt_sums))


    print("\nsums in broad classifications")
    [print(skeys[i], "sum %.f" %Tb_sums[i]) for i in range(len(skeys))]
    print("Total: ", sums(Tb_sums) )
    print("\nsums in narrow classifications")
    [print(ckeys[i], "sum %.f" %Fc_sums[i]) for i in range(len(ckeys))]
    print("Total: ", sums(Fc_sums ) )
    print("\nsums of transitions broad")
    T_eb = np.vstack([Nt_sums[0], 
                                 (Nt_sums[1] + Nt_sums[2]) , 
                                 (Nt_sums[3] + Nt_sums[4])])
    Nt_sums_broad = np.column_stack([T_eb[:, 0], 
                                 (T_eb[:, 1]+ T_eb[:, 2]), 
                                 (T_eb[:, 3]+ T_eb[:, 4])])
    print(Nt_sums_broad)
    print("\nsums of transitions narrow")
    print(Nt_sums)
    print("Total: ", sums(Nt_sums) )



    """print("\n%s in broad classifications")
    [print(skeys[i], " %.f%%" %Tb_percs[i]) for i in range(len(skeys))]
    print("\n%s in narrow classifications")
    [print(ckeys[i], " %.f%%" %Fc_percs[i]) for i in range(len(ckeys))]"""
    #ERROR ANALSIS HERE
    print("Errors in transition matrix (row_sums - column_sums / 2)")
    print((sums(Nt_percs_broad, axis = 1) - sums(Nt_percs_broad, axis = 1) / 2))
    
    print("Transition matrix, broad")
    T_eb = np.vstack([Nt_percs[0], 
                                 (Nt_percs[1] + Nt_percs[2]) , 
                                 (Nt_percs[3] + Nt_percs[4])])
    Nt_percs_broad = np.column_stack([T_eb[:, 0], 
                                 (T_eb[:, 1]+ T_eb[:, 2]), 
                                 (T_eb[:, 3]+ T_eb[:, 4])])
    print(Nt_percs_broad)
    print("row_sums of transitin matrix, broad")
    print(sums(Nt_percs_broad, axis = 1))
    print("\n%s of transitions broad(->bm, ->b, ->t):")
    #print("->bm %.f%%" %(Nt_percs[0, 0] + sums(Nt_percs[1:3, 0]) + sums(Nt_percs[3:5, 0])))
    print([sums(Nt_sums, axis = 0)[0]*100/sums(Nt_sums), 
           (sums(Nt_sums, axis = 0)[1] + sums(Nt_sums, axis = 0)[2])*100/(sums(Nt_sums)),
           (sums(Nt_sums, axis = 0)[3] + sums(Nt_sums, axis = 0)[4])*100/(sums(Nt_sums))])
    print("\n%s of transitions broad(bm->, b->, t->):")
    #print("->bm %.f%%" %(Nt_percs[0, 0] + sums(Nt_percs[1:3, 0]) + sums(Nt_percs[3:5, 0])))
    print([sums(Nt_sums, axis = 1)[0]*100/sums(Nt_sums),
           (sums(Nt_sums, axis = 1)[1] + sums(Nt_sums, axis = 1)[2])*100/(sums(Nt_sums)),
           (sums(Nt_sums, axis = 1)[3] + sums(Nt_sums, axis =1)[4])*100/(sums(Nt_sums))])#(Nt_percs[i,1] + Nt_percs[i,1+1] + Nt_percs[i+1, 1] + Nt_percs[i+1, 1+1] + sums(Nt_percs[0, 1:3]))) for i in [1,3]]
    #print("->t %.f%%" %(Nt_percs[i,2] + Nt_percs[i,2+1] + Nt_percs[i+1, 2] + Nt_percs[i+1, 2+1] + sums(Nt_percs[0, 3:5]))) for i in [1,3]]
    print("\nTransition matrix, narrow")
    print(Nt_percs)
    print(sums(Nt_percs))
    print("\n%s of transitions to states narrow:")
    [print(sums(Nt_percs, axis = 0)*100/sums(Nt_percs))]
    print("\n%s of transitions from states narrow:")
    [print(sums(Nt_percs, axis = 1)*100/sums(Nt_percs))]
    #print("\n%s of transitions narrow")
    #[print(tkeys[i], " %.f%%" %Nt_percs.flatten()[i]) for i in range(len(tkeys))]

In [ ]:
trans_runs= np.load('trans_runs.npy')[()]
num_top_keys = 4
num_full_keys = 6
num_tran_keys = 5
trans_threshes = [5, 8, 10, 19, 27, 38, 52, 60]
sides = {"bm":[], "b":[], "t":[], "ucf":[]}
classes = {"bm":[], "br":[], "bl":[], "tr":[], "tl":[], "ucf":[]}
transitions = {"bm->bm" :[], "bm->br":[], "bm->bl":[], "bm->tr":[], "bm->tl":[],
              "br->bm" :[], "br->br":[], "br->bl":[], "br->tr":[], "br->tl":[],
              "bl->bm" :[], "bl->br":[], "bl->bl":[], "bl->tr":[], "bl->tl":[],
              "tr->bm" :[], "tr->br":[], "tr->bl":[], "tr->tr":[], "tr->tl":[],
              "tl->bm" :[], "tl->br":[], "tl->bl":[], "tl->tr":[], "tl->tl":[]}

skeys = list(sides.keys())
ckeys = list(classes.keys())
for trans_fil in trans_threshes:
    print("\nTRANSITION THRESHOLD ", trans_fil)
    
    Top_bottoms, Full_classif, N_transitions = trans_runs[trans_fil]
    #Top_bottoms, Full_classif, N_transitions, trans_threshes, num_top_keys, num_full_keys,
                             #num_tran_keys, skeys, ckeys, num_tran_keys, num_total_frames = trans_runs[trans_fil]
    sums = np.sum
    Tb_sums = [sums(Top_bottoms[:, c]) for c in range(num_top_keys)]
    Fc_sums = [sums(Full_classif[:, c]) for c in range(num_full_keys)]
    Nt_sums = sums(N_transitions, axis = 0)#[sums(N_transitions[:, c]) for c in range(num_tran_keys)]
    #print("\nNumber of frames obbserved: ", num_total_frames)

    Tb_percs = [Tb_sums[c] *100 /sums(Tb_sums) for c in range(num_top_keys)]
    Fc_percs = [Fc_sums[c] *100/sums(Fc_sums) for c in range(num_full_keys)]
    Nt_percs = Nt_sums *100/sums(Nt_sums)
    #print(sums(N_transitions), sums(Nt_sums))


    print("\nsums in broad classifications")
    [print(skeys[i], "sum %.f" %Tb_sums[i]) for i in range(len(skeys))]
    print("Total: ", sums(Tb_sums) )
    print("\nsums in narrow classifications")
    [print(ckeys[i], "sum %.f" %Fc_sums[i]) for i in range(len(ckeys))]
    print("Total: ", sums(Fc_sums ) )
    print("\nsums of transitions broad")
    T_eb = np.vstack([Nt_sums[0], 
                                 (Nt_sums[1] + Nt_sums[2]) , 
                                 (Nt_sums[3] + Nt_sums[4])])
    Nt_sums_broad = np.column_stack([T_eb[:, 0], 
                                 (T_eb[:, 1]+ T_eb[:, 2]), 
                                 (T_eb[:, 3]+ T_eb[:, 4])])
    print(Nt_sums_broad)
    print("\nsums of transitions narrow")
    print(Nt_sums)
    print("Total: ", sums(Nt_sums) )



    print("\n%s in broad classifications")
    [print(skeys[i], " %.f%%" %Tb_percs[i]) for i in range(len(skeys))]
    print("\n%s in narrow classifications")
    [print(ckeys[i], " %.f%%" %Fc_percs[i]) for i in range(len(ckeys))]
    print("Transition matrix, broad")
    T_eb = np.vstack([Nt_percs[0], 
                                 (Nt_percs[1] + Nt_percs[2]) , 
                                 (Nt_percs[3] + Nt_percs[4])])
    Nt_percs_broad = np.column_stack([T_eb[:, 0], 
                                 (T_eb[:, 1]+ T_eb[:, 2]), 
                                 (T_eb[:, 3]+ T_eb[:, 4])])
    print(Nt_percs_broad)
    print("row_sums of transitin matrix, broad")
    print(sums(Nt_percs_broad, axis = 1))
    print("\n%s of transitions broad(->bm, ->b, ->t):")
    #print("->bm %.f%%" %(Nt_percs[0, 0] + sums(Nt_percs[1:3, 0]) + sums(Nt_percs[3:5, 0])))
    print([sums(Nt_sums, axis = 0)[0]*100/sums(Nt_sums), 
           (sums(Nt_sums, axis = 0)[1] + sums(Nt_sums, axis = 0)[2])*100/(sums(Nt_sums)),
           (sums(Nt_sums, axis = 0)[3] + sums(Nt_sums, axis = 0)[4])*100/(sums(Nt_sums))])
    print("\n%s of transitions broad(bm->, b->, t->):")
    #print("->bm %.f%%" %(Nt_percs[0, 0] + sums(Nt_percs[1:3, 0]) + sums(Nt_percs[3:5, 0])))
    print([sums(Nt_sums, axis = 1)[0]*100/sums(Nt_sums),
           (sums(Nt_sums, axis = 1)[1] + sums(Nt_sums, axis = 1)[2])*100/(sums(Nt_sums)),
           (sums(Nt_sums, axis = 1)[3] + sums(Nt_sums, axis =1)[4])*100/(sums(Nt_sums))])#(Nt_percs[i,1] + Nt_percs[i,1+1] + Nt_percs[i+1, 1] + Nt_percs[i+1, 1+1] + sums(Nt_percs[0, 1:3]))) for i in [1,3]]
    #print("->t %.f%%" %(Nt_percs[i,2] + Nt_percs[i,2+1] + Nt_percs[i+1, 2] + Nt_percs[i+1, 2+1] + sums(Nt_percs[0, 3:5]))) for i in [1,3]]
    print("\nTransition matrix, narrow")
    print(Nt_percs)
    print(sums(Nt_percs))
    print("\n%s of transitions to states narrow:")
    [print(sums(Nt_percs, axis = 0)*100/sums(Nt_percs))]
    print("\n%s of transitions from states narrow:")
    [print(sums(Nt_percs, axis = 1)*100/sums(Nt_percs))]
    #print("\n%s of transitions narrow")
    #[print(tkeys[i], " %.f%%" %Nt_percs.flatten()[i]) for i in range(len(tkeys))]